In [ ]:
import sklearn
from sklearn.cluster import KMeans
import numpy as np
from sklearn.svm import SVC
import pickle
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def VLAD(X,visualDictionary):

    predictedLabels = visualDictionary.predict(X)
    centers = visualDictionary.cluster_centers_
    labels=visualDictionary.labels_
    k=visualDictionary.n_clusters

    m,d = X.shape
    
    
    V=np.zeros([k,d])
    #computing the differences

    # for all the clusters (visual words)
    for i in range(k):
        # if there is at least one descriptor in that cluster
        if np.sum(predictedLabels==i)>0:
            # add the diferences
            V[i]=np.sum(X[predictedLabels==i,:]-centers[i],axis=0)


    V = V.flatten()
    # power normalization, also called square-rooting normalization
    V = np.sign(V)*np.sqrt(np.abs(V))

    # L2 normalization

    V = V/np.sqrt(np.dot(V,V))
    
    return V

In [ ]:
def get_vlad(doc,layer,scales,kmean1,kmean2,pca,type_set="train"):
    for n in scales:
        print(doc, n , layer)
        #laod_data
        if type_set=="train":
            x=np.load(dir_path_DataSet + load_flod + doc +'X_train_'+layer + '(' + str(n) + ').npy')
        elif type_set=="test":
            x=np.load(dir_path_DataSet + load_flod + doc +'X_test_'+layer + '(' + str(n) + ').npy')
            

        if layer=="block5_pool" or n!=0:
            #reshape (d1,d2,d3,d4) -> (d1,d2*d3,d4)
            x = np.reshape(x,(x.shape[0],(x.shape[1]*x.shape[2]),x.shape[3]))

            #if x_all is empty : x_all take x else add data to x_all 
            if(n==0):
                x_all=x

            else:
                x_all=np.concatenate((x_all,x),axis=1)
                
    
    V_64=[]
    V_256=[]
    #for ech image
    for vec in x_all:
        #we take the first vec and we delete it from the data set: memory problem
        x_all=x_all[1:,...]
        #normalization l2
        vec = normalize(vec, norm = 'l2', axis = 1)
        #transformation with PCA
        vec=pca.transform(vec)
        #normalisation
        vec = normalize(vec, norm = 'l2', axis = 1)
        
        
        V_64.append(VLAD(vec,kmean1))
        V_256.append(VLAD(vec,kmean2))

    
    return V_64,V_256
        

In [ ]:
load_flod='dataNpVGG/'
save_flod='fold_kvasir/'
#dataset's path 
dir_path_DataSet=''

ter_path=""
#fold name for saving files(kmean/PCA)
fold_name="files/"
vlad_fold="vlad_fold/"

docs=['miniMIT_Etus']#,'chest_xray','kvasir-dataset-v2']
scales=[0,1,2]
layers=['block5_pool']#,'block8_9_conv']#,'block8_8_conv']

In [ ]:
for doc in docs:
    for layer in layers:
        #load files 
        kmean1 = pickle.load(open(ter_path+fold_name+doc+'_'+layer+'_'+'kmean1.pickle', 'rb'))
        kmean2 = pickle.load(open(ter_path+fold_name+doc+'_'+layer+'_'+'kmean2.pickle', 'rb'))
        pca = pickle.load(open(ter_path+fold_name+doc+'_'+layer+'_'+'PCA.pickle', 'rb'))
        
        #lead file for train set
        v_train_64,v_train_256=get_vlad(doc,layer,scales,kmean1,kmean2,pca,type_set="train")
        v_train_64 = np.asarray(v_train_64)
        v_train_256= np.asarray(v_train_256)
        
        print(v_train_64.shape)
        np.save(ter_path+vlad_fold+doc+'_'+layer+'_'+'vlad_train(64).npy',v_train_64)
        np.save(ter_path+vlad_fold+doc+'_'+layer+'_'+'vlad_train(256).npy',v_train_256)
        del v_train_64,v_train_256
        
        #for test set
        v_test_64,v_test_256=get_vlad(doc,layer,scales,kmean1,kmean2,pca,type_set="test")
        
        v_test_64 = np.asarray(v_test_64)
        v_test_256= np.asarray(v_test_256)
        print(v_test_64.shape)
        #save vlad files
        np.save(ter_path+vlad_fold+doc+'_'+layer+'_'+'vlad_test(64).npy',v_test_64)
        np.save(ter_path+vlad_fold+doc+'_'+layer+'_'+'vlad_test(256).npy',v_test_256)
        
        print("done")